In [ ]:
# ========================================
# === Libraries ===
# ========================================
import pandas as pd
from IPython.display import display
import os


# DCSI

In [ ]:
### データの入力
INPUT_PATH_DCSI = "input dataのパス"
df = pd.read_csv(INPUT_PATH_DCSI, encoding='cp932')

### カラムの抽出
collist = [
"UID",
"Month",
"Year",
"Dealer",
"Region",
"REMINDER_",
"RECEPTION",
"CUSTOMER_LOUNGE",
"DELIVERY",
"REPAIR_QUALITY",
"FACILITY",
"Q30Y",
"OVERALL_SATISFACTION",
"Free_comment"
]

df = df[collist]

### データの出力
OUTPUT_PATH_DCSI = "output dataのパス"
df.to_csv(OUTPUT_PATH_DCSI, encoding='cp932', index=False)

# DCSIに関する追加の処理

In [ ]:
# データの入力
df = pd.read_csv("パスを入力してください", encoding='cp932')

# カラムの抽出・データ型の変換
df_DCSI = df[['UID', 'Q35Y']].copy()
df_DCSI['follow_call'] = pd.to_numeric(df_DCSI['Q35Y'], errors='coerce')

# follow_callごとの処理
df_DCSI.fillna({'follow_call': 0}, inplace=True)
df_DCSI_output = df_DCSI.groupby(by='UID').agg(
    follow_call=('follow_call', 'max')
).reset_index()

# ↑のセルで作成したデータの入力
df_DCSI_new_name = pd.read_csv("パスを入力してください")

# データの結合
df_DCSI_ver2 = pd.merge(df_DCSI_new_name, df_DCSI_output, on = 'UID', how = 'left')

# データの出力
df_DCSI_ver2.to_csv("パスを入力してください", encoding='cp932', index=False)

# My Honda

In [ ]:
### データの入力
INPUT_PATH_DCSI = "input dataのパス"
df = pd.read_csv(INPUT_PATH_DCSI, encoding='cp932')

### カラムの抽出
usecols = [
    'rate_service_point',
    'rate_mechanic_point',
    'rate_chatbot_point',
    'rate_chatbot_agent_point',
    'UID',
]

df_MH = df[usecols]


### 最後の点数を抽出
# rate_service_point, rate_mechanic_pointについて、小数表記のため3文字分を抽出
# rate_service_point
rate_service_point_5str_1 = df_MH.loc[:, 'rate_service_point'].str[-5:-1] 
rate_service_point_5str_2 = rate_service_point_5str_1.str[-3:]
# rate_mechanic_point
rate_mechanic_point_5str_1 = df_MH.loc[:, 'rate_mechanic_point'].str[-5:-1] 
rate_mechanic_point_5str_2 = rate_mechanic_point_5str_1.str[-3:]
# rate_chatbot_point, rate_chatbot_pointについて、整数表記のため1文字分を抽出
# rate_chatbot_point
rate_chatbot_point_5str_1 = df_MH.loc[:, 'rate_chatbot_point'].str[-2:-1] 
rate_chatbot_point_5str_2 = rate_chatbot_point_5str_1.str[-1:]
# rate_chatbot_agent_point
rate_chatbot_agent_point_5str_1 = df_MH.loc[:, 'rate_chatbot_agent_point'].str[-5:-1] 
rate_chatbot_agent_point_5str_2 = rate_chatbot_agent_point_5str_1.str[-1:]


### 文字列から数値への型変換
# rate_service_point
df_rate_service_point_num = pd.to_numeric(rate_service_point_5str_2, errors='coerce') 
# rate_mechanic_point
df_rate_mechanic_point_num =  pd.to_numeric(rate_mechanic_point_5str_2, errors='coerce')
# rate_chatbot_point
df_rate_chatbot_point_num =  pd.to_numeric(rate_chatbot_point_5str_2, errors='coerce')
# rate_chatbot_agent_point
df_rate_chatbot_agent_point_num =  pd.to_numeric(rate_chatbot_agent_point_5str_2, errors='coerce')
### dataframeへの変換
df_rate_service_point_num = pd.DataFrame(df_rate_service_point_num)
df_rate_mechanic_point_num =  pd.DataFrame(df_rate_mechanic_point_num)
df_rate_chatbot_point_num =  pd.DataFrame(df_rate_chatbot_point_num)
df_rate_chatbot_agent_point_num =  pd.DataFrame(df_rate_chatbot_agent_point_num)


### - カラム名の変更とカラム抽出
# - rate_service_point
df_rate_service_point_num.rename(columns={
    'rate_service_point' : 'rate_service_point_NUM'
}, inplace=True)
# - rate_mechanic_point
df_rate_mechanic_point_num.rename(columns={
    'rate_mechanic_point' : 'rate_mechanic_point_NUM'
}, inplace=True)
# - rate_chatbot_point
df_rate_chatbot_point_num.rename(columns={
    'rate_chatbot_point' : 'rate_chatbot_point_NUM'
}, inplace=True)
# - rate_chatbot_agent_point
df_rate_chatbot_agent_point_num.rename(columns={
    'rate_chatbot_agent_point': 'rate_chatbot_agent_point_NUM'
}, inplace=True)
### データの結合
df_MH2 = pd.concat([df_MH, df_rate_service_point_num, df_rate_mechanic_point_num, df_rate_chatbot_point_num, df_rate_chatbot_agent_point_num], axis = 'columns')
### 必要カラムの抽出
df_MH2.drop(columns=['rate_service_point', 'rate_mechanic_point', 'rate_chatbot_point', 'rate_chatbot_agent_point'], inplace=True)
# カラム名の編集
df_MH2.rename(columns={
    'rate_service_point_NUM' : 'rate_service_point',
    'rate_mechanic_point_NUM' : 'rate_mechanic_point',
    'rate_chatbot_point_NUM' : 'rate_chatbot_point',
    'rate_chatbot_agent_point_NUM' : 'rate_chatbot_agent_point',
}, inplace=True)
# 必要カラムの抽出
df_MH3 = df_MH2[['UID',  'rate_service_point', 'rate_mechanic_point', 'rate_chatbot_point', 'rate_chatbot_agent_point']]

### uniqueなUIDごとに計算する
# uniqueなUID（昇順）の取得
df_uid = df_MH3['UID']
uid_unique = df_uid.unique()
df_uid_unique = pd.DataFrame({'UID': uid_unique})

# UIDの出現回数をカウント
df_uid_count = df_uid.value_counts().rename_axis('UID').reset_index(name='UID_counts_MyHonda')

# uniqueなuid（昇順）に出現回数を付与
df_uid_unique_count = pd.merge(df_uid_unique, df_uid_count, on='UID', how='left')

# UIDごとに、各pointの平均をとる
df_MyHonda = df_MH2.groupby(['UID'], dropna=False, as_index=False).agg(
        rate_service_point = ('rate_service_point', 'mean'),
        rate_mechanic_point = ('rate_mechanic_point', 'mean'),
        rate_chatbot_point = ('rate_chatbot_point', 'mean'),
        rate_chatbot_agent_point = ('rate_chatbot_agent_point', 'mean'),
)
# uniqueなUIDに結合
df_MyHonda = pd.merge(df_uid_unique_count, df_MyHonda, on='UID', how='left')

# MyhondaID有無を表すカラム：UIIDがあればMyHondaIDありとする
df_MyHonda.loc[:, ["MyHonda有無"]] = '1'

### データの出力
OUTPUT_PATH_MH = "output path"
df_MyHonda.to_csv(OUTPUT_PATH_MH, encoding='cp932', index=False)

# booking_service

In [ ]:
# UIDから出現回数と有無を表すフラグを立てる関数
def uid_unique_count_flag(df, filename):
    # df_uid: UIDのみのタプル
    df_uid = df['UID']

    # uniqueなUIDの取得
    uid_unique = df_uid.unique()
    df_uid_unique = pd.DataFrame({'UID': uid_unique})
    
    # UIDの出現回数
    column_name = 'UID_counts_' + filename
    df_uid_count = df_uid.value_counts().rename_axis('UID').reset_index(name=column_name)
    
    # uniqueなuidと出現回数のカラムの結合
    df_uid_unique_count = pd.merge(df_uid_unique, df_uid_count, on='UID', how='left')
    
    # UID有無を表すカラム
    df_uid_unique_count.loc[:, ["UID有無"]] = '1'

    return df_uid_unique_count

In [ ]:
### データの入力
INPUT_PATH_MH = "input path"
df = pd.read_csv(INPUT_PATH_MH, encoding='cp932')
df_book = df.iloc[:, [-1, 1, 3, 5, 13]]
df_book_uid = uid_unique_count_flag(df_book, 'book')

### emp_idについての処理
df_book["emp_id_num"] = pd.to_numeric(df_book['emp_id'], errors='coerce')
df_book.loc[~df_book['emp_id_num'].isna(), ['emp_choice']] = 1
df_book.loc[df_book['emp_id_num'].isna(), ['emp_choice']] = 0
df_emp = df_book.groupby(by='UID').agg(
    emp_id_count=('emp_choice', 'sum'),
    emp_id_有無=('emp_choice', 'max'),
).reset_index()

### booking_dateについての処理
df_booking_date = df_book.groupby(by="UID").aggregate(
    booking_date_count = ('booking_date', 'nunique'),
).reset_index()


### 結合
df_book_uid = pd.merge(df_book_uid, df_emp, on = 'UID', how='left')
df_book_uid = pd.merge(df_book_uid, df_booking_date, on = 'UID', how='left')

### データの出力
OUTPUT_PATH_book = "output path"
df_book_uid.to_csv(OUTPUT_PATH_book, encoding='cp932', index=False)

# notificiation_tracking, event_tracking
- 処理ファイル一覧
    - event_v_promo.csv
    - notif_vpi
    - notif_vpm
    - notif_v_pro
    - notif_v_spa
    - notif_book_pi
    - notif_book_pm
    - notif_book_spa

In [ ]:
# UIDから出現回数と有無を表すフラグを立てる関数
def uid_unique_count_flag(df, filename):
    # df_uid: UIDのタプル
    df_uid = df['UID']

    # uniqueなUIDの取得
    uid_unique = df_uid.unique()
    df_uid_unique = pd.DataFrame({'UID': uid_unique})

    # UIDの出現回数
    column_name = 'UID_counts_' + filename
    df_uid_count = df_uid.value_counts().rename_axis('UID').reset_index(name=column_name)
    
    # uniqueなuidと出現回数のカラムの結合
    df_uid_unique_count = pd.merge(df_uid_unique, df_uid_count, on='UID', how='left')

    # UID有無を表すカラム
    df_uid_unique_count.loc[:, ["UID有無"]] = '1'

    return df_uid_unique_count

In [ ]:
### 入力パス、出力パスの指定
PATH_DIR = "input dirを指定してください"
OUTPUT_PATH_DIR = "output dirを指定してください"

input_filename_list = [
    "event_v_promo",
    "notif_vpi",
    "notif_vpm",
    "notif_v_pro",
    "notif_v_spa",
    "notif_book_pi",
    "notif_book_pm",
    "notif_book_spa",
]

input_path_list = []
output_path_list = []

for name in input_filename_list:
    # input
    name_csv = name + '.csv'
    input_path = os.path.join(PATH_DIR, name_csv)
    input_path_list.append(input_path)

    # output
    output_name = name_csv.replace(".csv", "_output.csv")
    output_path = os.path.join(OUTPUT_PATH_DIR, output_name)
    output_path_list.append(output_path)

### 各ファイルについて処理
for input, output, filename in zip(input_path_list, output_path_list, input_filename_list):
    df_input = pd.read_csv(input, encoding="cp932")
    df_output = uid_unique_count_flag(df_input, filename)
    df_output.to_csv(output, encoding="cp932", index=False)


In [ ]:
### notif_v_pro, event_v_promoについて、個別に対応
df_notif = "↑のセルで生成したファイルのパスを個別に指定"
df_event = "↑のセルで生成したファイルのパスを個別に指定"

# SCA

In [ ]:
### データの入力
INPUT_PATH_MH = "データの入力パスを指定してください"
df = pd.read_csv(INPUT_PATH_MH, encoding='cp932')
df['start_date_df'] = pd.to_datetime(df['start_date'])

### カラムの抽出
df_sca_input = df[['UID', 'start_date_df']]

### UIDごと
df_sca = df_sca_input.groupby(by='UID').agg(
    sca_visit_num=('start_date_df', 'count'),
).reset_index()

### 出力
OUTPUT_PATH = "データの出力パスを指定してください"
df_sca.to_csv(OUTPUT_PATH, encoding='cp932', index=False)

# Comp

In [ ]:
### データの入力
INPUT_PATH_MH = "データの入力パスを指定してください"
df = pd.read_csv(INPUT_PATH_MH, encoding='cp932')
df['start_date_df'] = pd.to_datetime(df['start_date'])

### カラムの抽出
df_comp_input = df.loc[:, ['UID', 'CR Code Full']]
df_comp_input['suggestion'] = 0


# CR code full（提案）に属するフラグを立てる
list_suggestion = ["CA99", " CB98", " CB99", " CC99", " CD99", " CE99", " CF99", " CZ99", " CZ80", " CZ81"]
df_comp_input.loc[df_comp_input["CR Code Full"].isin(list_suggestion), ['suggestion']] = 1 


### UIDごとの提案回数をカウント
df_comp = df_comp_input.groupby(by='UID').agg(
    CR_feedback=('suggestion', 'count'),
).reset_index()
df_comp['CR'] = '1'

### 出力
OUTPUT_PATH = "データの出力パスを指定してください"
df_comp.to_csv(OUTPUT_PATH, encoding='cp932', index=False)


# ew

In [ ]:
### データの入力
INPUT_PATH_MH = "データの入力パスを指定してください"
df = pd.read_csv(INPUT_PATH_MH, encoding='cp932')
df['start_date_df'] = pd.to_datetime(df['start_date'])

### カラムの抽出・カラム名の修正
df_ew = df.iloc[:, [-1, 4, 11]].copy()
# カラム名の修正
df_ew.rename(columns={
    'Lo<U+1EA1>i goi' : 'Package_Type'
}, inplace=True)

### UIDごとの算出
df_ew_NewTotal = df_ew.groupby(by='UID').agg(
    ew_sales=('NET', 'sum'),
).reset_index()

### ALL_carフラグを立てて算出
df_ew['ALL_car'] = 0
df_ew.loc[df_ew["Package_Type"]=='All car', ['ALL_car']] = 1 
df_ew_Allcar = df_ew.groupby(by='UID').agg(
    ew_all_car = ('ALL_car', 'max'),
    ew_nun = ('ALL_car', 'sum')
)

### 結合
df_ew_output = pd.merge(df_ew_NewTotal, df_ew_Allcar, on='UID', how = 'outer')

### 出力
OUTPUT_PATH = "データの出力パスを指定してください"
df_ew.to_csv(OUTPUT_PATH, encoding='cp932', index=False)


# chat

In [ ]:
# データの入力
INPUT_PATH = "パスを入力してください"
collist = [10, 5, 6]
df = pd.read_csv(INPUT_PATH, usecols=collist, encoding='cp932')

### UIDの出現回数のカウント
df_uid_count = df['UID'].value_counts().rename_axis('UID').reset_index(name='chat_num')

### Service Partについて
# Business=AMかつField=Servideのレコードにフラグ1、その他を0として→agg max
df.loc[(df['Business'] == 'AM') & (df['Field'] == 'Service'), ['Business_Field']] = 1
df.loc[~((df['Business'] == 'AM') & (df['Field'] == 'Service')), ['Business_Field']] = 0
df_service_part = df.groupby(by='UID').agg(
    service_part = ('Business_Field', 'max')
).reset_index()

### 結合
df_chat = pd.merge(df_uid_count, df_service_part, on = 'UID', how='left')

# UID有無を表す列を作成
df_chat.loc[:, 'chat'] = 1

# 必要カラムの抽出
df_chat_output = df_chat[['UID', 'chat','chat_num' , 'service_part']]

# データの出力
OUTPUT_PATH = os.path.join(PATH_DIR, "chat_output.csv")
df_chat.to_csv(OUTPUT_PATH, index=False, encoding=False)
# test_drive
PATH_DIR = "C:/Users/J0135690/OneDrive - Honda/ドキュメント/プロジェクト/CeDX/データ"
INPUT_PATH = os.path.join(PATH_DIR, "test_drive.csv")
OUTPUT_PATH = os.path.join(PATH_DIR, "test_drive_output.csv")

# test_drive

In [ ]:
### データの入力
INPUT_PATH = "パスを入力してください"
df = pd.read_csv("パスを入力してください", encoding='cp932')

# UIDごとにevent_date回数をカウント
df_test_drive = df.groupby(by='UID').agg(
    test_drive = ('event_date', 'count')
).reset_index()

### データの出力
df_test_drive.to_csv("パスを入力してください", encoding='cp932', index=False)

# view_test_drive

In [ ]:
### データの入力
df = pd.read_csv("パスを入力してください", encoding='cp932')

# UIDごとにevent_date回数をカウント
df_view_test_drive = df.groupby(by='UID').agg(
    view_test_drive=('event_date', 'count')
).reset_index()

### データの出力
df_view_test_drive.to_csv("パスを入力してください", encoding='cp932', index=False)

# view_promotion_notification

In [ ]:
# データの入力
INPUT_PATH = "パスを入力してください"
df = pd.read_csv(INPUT_PATH, encoding='cp932')

# UIDごとにevent_date回数をカウント
df_view_promotion_notification = df.groupby(by='UID').agg(
    view_promotion_notification=('event_date', 'count')
).reset_index()

# データの出力
OUTPUT_PATH = "パスを入力してください"
df_view_promotion_notification.to_csv(OUTPUT_PATH, encoding='cp932', index=False)

# compare_product, estimate_cost, view_productについて

- 入力したファイル一覧
    - compare_product,
    - estimate_cost,
    - view_product_detail_am,
    - view_product_gallery,
    - view_product_list,
    - view_product_color

In [ ]:
def yymmdd_count(df, s):
    df_output = df.groupby(by='UID').agg(
        s = ('event_date', 'count')
    ).reset_index()
    return df_output

In [ ]:
### 入力パス、出力パスの指定
PATH_DIR = "input dirを指定してください"
OUTPUT_PATH_DIR = "output dirを指定してください"

INPUT_FILENAME_LIST = [
    'compare_product_am',
    'estimate_cost',
    'view_product_detail_am',
    'view_product_gallery',
    'view_product_list',
    'view_product_color'
]

input_path_list = []
output_path_list = []

for name in INPUT_FILENAME_LIST:
    # input
    name_csv = name + '.csv'
    input_path = os.path.join(PATH_DIR, name_csv)
    input_path_list.append(input_path)

    # output
    output_name_csv = name_csv.replace(".csv", "_output.csv")
    output_path = os.path.join(OUTPUT_PATH_DIR, output_name_csv)
    output_path_list.append(output_path)

### 各ファイルについて処理
for input, output, filename in zip(input_path_list, output_path_list, INPUT_FILENAME_LIST):
    df_input = pd.read_csv(input, encoding="cp932")
    df_output = yymmdd_count(df_input, filename)
    df_output.to_csv(output, encoding="cp932", index=False)